In [1]:
import os
import shutil

# 设置路径
data_dir = './videos/bed_rest_MRI'
data_dir = './videos/AFL_MRI'

label_dir = os.path.join(data_dir, 'label_in_png_L4L5_4class_DINO_order')
# prompt_dir = 'manual_prompt_frame0_1_2'
# prompt_dir = 'manual_prompt_frame0_1'
prompt_dir = 'manual_prompt_frame0'

auto_seg_dir = os.path.join(data_dir, prompt_dir, 'SAM2_seg_mask_nolap')


# 创建新的目标目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')
os.makedirs(output_dir, exist_ok=True)

# 获取 label_dir 中的文件名映射
label_mapping = {}
for label_file in os.listdir(label_dir):
    if label_file.endswith('.png'):
        file_prefix = label_file.split('_')[0]  # 获取开头的00000, 00001等
        label_mapping[file_prefix] = label_file

# 复制并重命名文件
for file in os.listdir(auto_seg_dir):
    if 'combined.png' in file:
        # 提取 frame_xx 编号
        frame_idx = file.split('_')[1]  # 获取 frame_xx 中的 xx
        frame_prefix = frame_idx.zfill(5)  # 确保前导零一致 (00000, 00001)

        # 查找匹配的 label 文件名
        if frame_prefix in label_mapping:
            new_filename = label_mapping[frame_prefix]
            src_path = os.path.join(auto_seg_dir, file)
            dst_path = os.path.join(output_dir, new_filename)

            # 复制并重命名文件
            shutil.copy(src_path, dst_path)
            print(f"Copied {file} → {new_filename}")

print(f"All files copied to {output_dir}")


Copied frame_63_combined.png → 00063_ID2B_R0_slice_026_imgseg_.png
Copied frame_34_combined.png → 00034_ID12M_BDC_slice_013_imgseg_.png
Copied frame_54_combined.png → 00054_ID5E_BDC_slice_018_imgseg_.png
Copied frame_0_combined.png → 00000_ID4D_BDC_slice_017_imgseg_.png
Copied frame_62_combined.png → 00062_ID24Z_R13_slice_028_imgseg_.png
Copied frame_61_combined.png → 00061_ID11L_R0_slice_025_imgseg_.png
Copied frame_20_combined.png → 00020_ID1A_R13_slice_020_imgseg_.png
Copied frame_19_combined.png → 00019_ID4D_R13_slice_025_imgseg_.png
Copied frame_69_combined.png → 00069_ID22X_R0_slice_025_imgseg_.png
Copied frame_64_combined.png → 00064_ID17S_BDC_slice_020_imgseg_.png
Copied frame_4_combined.png → 00004_ID19U_BDC_slice_026_imgseg_.png
Copied frame_11_combined.png → 00011_ID11L_R13_slice_016_imgseg_.png
Copied frame_13_combined.png → 00013_ID15Q_BDC_slice_018_imgseg_.png
Copied frame_51_combined.png → 00051_ID13N_R13_slice_030_imgseg_.png
Copied frame_8_combined.png → 00008_ID2B_R13

In [2]:
import os, glob,cv2
import SimpleITK as sitk
import numpy as np
from PIL import Image  # 用于读取 PNG 掩码

# -------------- 保存 NIfTI 文件的函数 --------------
def save_array_to_nii(mask_array, nii_template_path, nii_save_path):
    """
    将 NumPy 掩码数组保存为 NIfTI 格式，保持与模板 NIfTI 文件相同的空间信息。
    """
    sitk_image_object = sitk.ReadImage(nii_template_path)
    output_spacing = sitk_image_object.GetSpacing()
    output_direction = sitk_image_object.GetDirection()
    output_origin = sitk_image_object.GetOrigin()

    # 将 NumPy 数组转换为 SimpleITK 图像对象
    nii_output = sitk.GetImageFromArray(mask_array)
    nii_output.SetSpacing(output_spacing)
    nii_output.SetDirection(output_direction)
    nii_output.SetOrigin(output_origin)

    # 保存 NIfTI 文件
    sitk.WriteImage(nii_output, nii_save_path)
    print(f"Saved: {nii_save_path}")

# -------------- 目录路径 --------------
data_dir = './videos/bed_rest_MRI'
label_nii_dir = os.path.join(data_dir, 'data_in_nii/label_in_nii_72')  # 原始 NIfTI 文件
seg_nii_dir = os.path.join(data_dir,  prompt_dir, 'seg_in_nii_72_L4-L5')  # 保存分割 NIfTI 的目录
output_dir = os.path.join(data_dir, prompt_dir, 'seg_mask_renamed')  # 自动分割 PNG 掩码所在目录

# seg_nii_dir = os.path.join(data_dir, 'label_in_nii_72_L4-L5')  # 保存分割 NIfTI 的目录
# output_dir = os.path.join(data_dir, 'data_in_nii','label_in_png_L4L5_4class_KMeans_sorted')  # 自动分割 PNG 掩码所在目录


os.makedirs(seg_nii_dir, exist_ok=True)  # 创建保存路径（如果不存在）

# -------------- 遍历自动分割的掩码文件 --------------
for idx, filename in enumerate(os.listdir(output_dir)):    
    print(f"{idx}: Processing {filename}")

    # 拆分文件名，提取 `pid` 和 `slice_idx`
    parts = filename.split('_')
    pid = "_".join(parts[1:-4])  # 提取 PID，例如 "ID1A_R0"
    slice_idx = int(parts[-3].replace('slice', ''))  # 提取 Slice Index，例如 017 → 17

    # 查找匹配的 NIfTI 文件
    nii_files = glob.glob(os.path.join(label_nii_dir, pid + "*.nii.gz"))
    if not nii_files:
        print(f"Warning: No matching NIfTI file found for PID {pid}")
        continue
    label_nii_path = nii_files[0]

    # 读取 label NIfTI 作为模板
    label_sitk = sitk.ReadImage(label_nii_path)
    label_array = sitk.GetArrayFromImage(label_sitk)  # 转换为 NumPy 数组
    
    # 创建新的 mask_array（与 label_array 形状相同，全为 0）
    mask_array = np.zeros_like(label_array, dtype=np.uint8)

    # 读取 PNG 掩码文件（auto_mask）
    mask_path = os.path.join(output_dir, filename)
    auto_mask = np.array(Image.open(mask_path))

   # 获取 mask_array[slice_idx, :, :] 的目标形状
    target_shape = mask_array[slice_idx, :, :].shape  # (H, W)

    # 初始化新的 mask_mapped_resized 数组（与目标形状相同）
    mask_mapped_resized = np.zeros(target_shape, dtype=np.uint8)

    # 逐个处理每个 class，进行 resize 并合并
    for class_value in [50, 100, 150, 200]:
        binary_mask = (auto_mask == class_value).astype(np.uint8)  # 生成二值 mask
        resized_binary_mask = cv2.resize(binary_mask, (target_shape[1], target_shape[0]), interpolation=cv2.INTER_NEAREST)  # 保持最近邻插值
        mask_mapped_resized[resized_binary_mask > 0] = {50: 1, 100: 2, 150: 3, 200: 4}[class_value]  # 赋值回正确的 class

    # 赋值到 mask_array
    if 'ID' in pid:
        mask_mapped_resized =np.rot90(mask_mapped_resized, 2) 
    mask_array[slice_idx, :, :] = mask_mapped_resized

    # 设置保存路径
    nii_save_path = os.path.join(seg_nii_dir, f"{pid}_L4-L5_sam2_seg_slice{slice_idx}_seg.nii.gz")

    # 保存 NIfTI 掩码文件
    save_array_to_nii(mask_array, label_nii_path, nii_save_path)

    print(f"PID: {pid}, Slice Index: {slice_idx}")
    print(f"Label NIfTI Path: {label_nii_path}")
    print(f"Saved Segmentation: {nii_save_path}\n\n")


0: Processing 00023_ID1A_R0_slice_017_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID1A_R0_L4-L5_sam2_seg_slice17_seg.nii.gz
PID: ID1A_R0, Slice Index: 17
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID1A_R0_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID1A_R0_L4-L5_sam2_seg_slice17_seg.nii.gz


1: Processing 00017_ID8H_R13_slice_028_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID8H_R13_L4-L5_sam2_seg_slice28_seg.nii.gz
PID: ID8H_R13, Slice Index: 28
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID8H_R13_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID8H_R13_L4-L5_sam2_seg_slice28_seg.nii.gz


2: Processing 00016_ID8H_BDC_slice_020_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID8H_BDC_L4-L5_sam2_seg_slic

Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID9J_R13_L4-L5_sam2_seg_slice39_seg.nii.gz
PID: ID9J_R13, Slice Index: 39
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID9J_R13_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID9J_R13_L4-L5_sam2_seg_slice39_seg.nii.gz


20: Processing 00050_ID13N_R0_slice_035_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID13N_R0_L4-L5_sam2_seg_slice35_seg.nii.gz
PID: ID13N_R0, Slice Index: 35
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID13N_R0_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID13N_R0_L4-L5_sam2_seg_slice35_seg.nii.gz


21: Processing 00065_ID17S_R13_slice_026_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID17S_R13_L4-L5_sam2_seg_slice26_seg.nii.gz
PID: ID17S_R13, Slice Index

Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID7G_BDC_L4-L5_sam2_seg_slice22_seg.nii.gz
PID: ID7G_BDC, Slice Index: 22
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID7G_BDC_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID7G_BDC_L4-L5_sam2_seg_slice22_seg.nii.gz


40: Processing 00013_ID15Q_BDC_slice_018_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID15Q_BDC_L4-L5_sam2_seg_slice18_seg.nii.gz
PID: ID15Q_BDC, Slice Index: 18
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID15Q_BDC_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID15Q_BDC_L4-L5_sam2_seg_slice18_seg.nii.gz


41: Processing 00011_ID11L_R13_slice_016_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID11L_R13_L4-L5_sam2_seg_slice16_seg.nii.gz
PID: ID11L_R13, Slice 

Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID10K_BDC_L4-L5_sam2_seg_slice42_seg.nii.gz
PID: ID10K_BDC, Slice Index: 42
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID10K_BDC_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID10K_BDC_L4-L5_sam2_seg_slice42_seg.nii.gz


59: Processing 00018_ID4D_R0_slice_017_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID4D_R0_L4-L5_sam2_seg_slice17_seg.nii.gz
PID: ID4D_R0, Slice Index: 17
Label NIfTI Path: ./videos/bed_rest_MRI/data_in_nii/label_in_nii_72/ID4D_R0_Water_seg_by_zhongyi.nii.gz
Saved Segmentation: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID4D_R0_L4-L5_sam2_seg_slice17_seg.nii.gz


60: Processing 00057_ID3C_R0_slice_023_imgseg_.png
Saved: ./videos/bed_rest_MRI/manual_prompt_frame0/seg_in_nii_72_L4-L5/ID3C_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
PID: ID3C_R0, Slice Index: 23
La

In [3]:
import os
import glob
import SimpleITK as sitk
import numpy as np
import pandas as pd  # 用于保存 CSV
from scipy.ndimage import label

# -------------- 目录 --------------
data_dir = './videos/bed_rest_MRI'
seg_nii_dir = os.path.join(data_dir,  prompt_dir, 'seg_in_nii_72_L4-L5')  # 分割掩码
# seg_nii_dir = os.path.join(data_dir, 'label_in_nii_72_L4-L5')  # 分割掩码

water_nii_dir = os.path.join(data_dir, 'data_in_nii', 'img_in_nii_72')  # 水图像
fat_nii_dir = os.path.join(data_dir, 'data_in_nii', 'img_in_nii_fat')  # 脂肪图像

# 创建 DataFrame 存储结果
df_results = pd.DataFrame(columns=["PID", "Class", "Mean Fat Ratio", "Std Fat Ratio", "Pixel Count"])

# -------------- 遍历分割掩码文件 --------------
for idx, filename in enumerate(os.listdir(seg_nii_dir)):
    print(f"{idx}: Processing {filename}")

    # 提取 PID
    parts = filename.split('_')
    pid = "_".join(parts[0:2])  # 例如 "ID1A_R0"

    # 查找对应的水图像和脂肪图像
    water_nii_files = glob.glob(os.path.join(water_nii_dir, pid + "*.nii.gz"))
    fat_nii_files = glob.glob(os.path.join(fat_nii_dir, pid + "*.nii.gz"))
    
    if not water_nii_files or not fat_nii_files:
        print(f"Warning: Missing water/fat NIfTI for {pid}")
        continue

    water_nii_path = water_nii_files[0]
    fat_nii_path = fat_nii_files[0]

    # 读取 NIfTI 数据
    mask_sitk = sitk.ReadImage(os.path.join(seg_nii_dir, filename))
    mask_array = sitk.GetArrayFromImage(mask_sitk)

    water_sitk = sitk.ReadImage(water_nii_path)
    water_nii_array = sitk.GetArrayFromImage(water_sitk)

    fat_sitk = sitk.ReadImage(fat_nii_path)
    fat_nii_array = sitk.GetArrayFromImage(fat_sitk)

    # 确保形状匹配
    assert mask_array.shape == water_nii_array.shape == fat_nii_array.shape, "Shape mismatch between mask and images!"

    # -------------- 计算每个类别的脂肪比率 --------------
    class_ratios = {}  # 存储每个类别的脂肪比率
    for class_value in [1, 2, 3, 4]:  # 分别计算 4 个类别
        class_mask = (mask_array == class_value)  # 选取当前类别的区域
        class_mask = class_mask >0 
        
        if np.sum(class_mask) == 0:
            continue  # 如果该类别没有像素，则跳过
            
        # -------------- 找到最大连通区域 --------------
        labeled_mask, num_features = label(class_mask)  # 计算连通区域
        if num_features > 1:  # 如果有多个连通区域
            region_sizes = [(labeled_mask == i).sum() for i in range(1, num_features + 1)]  # 计算每个区域的大小
            max_region_idx = np.argmax(region_sizes) + 1  # 找到最大连通区域的索引（注意索引从 1 开始）
            class_mask = labeled_mask == max_region_idx  # 仅保留最大连通区域

               # 计算脂肪比率（fat ratio map）
        class_mask = class_mask > 0  # 确保 mask 是布尔值

        # 提取 ROI 内的 fat 和 water 信号
        fat_values = fat_nii_array[class_mask]
        water_values = water_nii_array[class_mask]
        print(fat_values.shape)

        # 计算脂肪比率，避免除零
        epsilon = 1e-6
        fat_ratio_map = fat_values / (fat_values + water_values + epsilon)  # fat_ratio_map 现在是一维数组

        # 计算均值和标准差（不需要再用 class_mask 进行索引）
        class_ratios[class_value] = {
            "mean": np.mean(fat_ratio_map),  # 直接计算均值
            "std": np.std(fat_ratio_map),    # 直接计算标准差
            "count": len(fat_ratio_map),     # 计算 ROI 内的体素数
        }


    # -------------- 输出并写入 DataFrame --------------
    print(f"PID: {pid}")
    for class_value, stats in class_ratios.items():
        print(f"  Class {class_value}: Mean Fat Ratio = {stats['mean']:.4f}, Std = {stats['std']:.4f}, Count = {stats['count']}")

        # 将数据追加到 DataFrame
        df_results = df_results._append({
            "PID": pid,
            "Class": class_value,
            "Mean Fat Ratio": stats["mean"],
            "Std Fat Ratio": stats["std"],
            "Pixel Count": stats["count"]
        }, ignore_index=True)

    print("\n")

# -------------- 保存 CSV --------------
csv_save_path = os.path.join(data_dir,  prompt_dir, prompt_dir+"fat_ratio_results.csv")
df_results.to_csv(csv_save_path, index=False)
print(f"Results saved to {csv_save_path}")


0: Processing ID9J_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
(1349,)
(983,)
(958,)
(1465,)
PID: ID9J_R0
  Class 1: Mean Fat Ratio = 0.1938, Std = 0.2393, Count = 1349
  Class 2: Mean Fat Ratio = 0.0730, Std = 0.1169, Count = 983
  Class 3: Mean Fat Ratio = 0.0764, Std = 0.1236, Count = 958
  Class 4: Mean Fat Ratio = 0.2308, Std = 0.2532, Count = 1465


1: Processing ID24Z_BDC_L4-L5_sam2_seg_slice15_seg.nii.gz
(1510,)


/tmp/ipykernel_26939/3680783185.py:93: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = df_results._append({


(1232,)
(746,)
(825,)
PID: ID24Z_BDC
  Class 1: Mean Fat Ratio = 0.4748, Std = 0.2629, Count = 1510
  Class 2: Mean Fat Ratio = 0.2346, Std = 0.2186, Count = 1232
  Class 3: Mean Fat Ratio = 0.1813, Std = 0.2176, Count = 746
  Class 4: Mean Fat Ratio = 0.4099, Std = 0.2535, Count = 825


2: Processing ID10K_BDC_L4-L5_sam2_seg_slice42_seg.nii.gz
(793,)
(905,)
(915,)
(635,)
PID: ID10K_BDC
  Class 1: Mean Fat Ratio = 0.3689, Std = 0.2722, Count = 793
  Class 2: Mean Fat Ratio = 0.0743, Std = 0.0736, Count = 905
  Class 3: Mean Fat Ratio = 0.0948, Std = 0.1488, Count = 915
  Class 4: Mean Fat Ratio = 0.1990, Std = 0.1990, Count = 635


3: Processing ID12M_BDC_L4-L5_sam2_seg_slice13_seg.nii.gz
(1530,)
(639,)
(702,)
(1128,)
PID: ID12M_BDC
  Class 1: Mean Fat Ratio = 0.2274, Std = 0.2492, Count = 1530
  Class 2: Mean Fat Ratio = 0.0780, Std = 0.1037, Count = 639
  Class 3: Mean Fat Ratio = 0.1003, Std = 0.1424, Count = 702
  Class 4: Mean Fat Ratio = 0.1400, Std = 0.1612, Count = 1128


4: Pr

(922,)
(931,)
(1028,)
(725,)
PID: ID15Q_R13
  Class 1: Mean Fat Ratio = 0.2540, Std = 0.2243, Count = 922
  Class 2: Mean Fat Ratio = 0.0613, Std = 0.0758, Count = 931
  Class 3: Mean Fat Ratio = 0.1055, Std = 0.1831, Count = 1028
  Class 4: Mean Fat Ratio = 0.3694, Std = 0.2784, Count = 725


26: Processing ID3C_R0_L4-L5_sam2_seg_slice23_seg.nii.gz
(1782,)
(892,)
(889,)
(1425,)
PID: ID3C_R0
  Class 1: Mean Fat Ratio = 0.3368, Std = 0.2912, Count = 1782
  Class 2: Mean Fat Ratio = 0.1646, Std = 0.1925, Count = 892
  Class 3: Mean Fat Ratio = 0.2067, Std = 0.2189, Count = 889
  Class 4: Mean Fat Ratio = 0.3601, Std = 0.3021, Count = 1425


27: Processing ID12M_R0_L4-L5_sam2_seg_slice19_seg.nii.gz
(1362,)
(542,)
(553,)
(954,)
PID: ID12M_R0
  Class 1: Mean Fat Ratio = 0.3302, Std = 0.3197, Count = 1362
  Class 2: Mean Fat Ratio = 0.0897, Std = 0.1072, Count = 542
  Class 3: Mean Fat Ratio = 0.1243, Std = 0.1763, Count = 553
  Class 4: Mean Fat Ratio = 0.2440, Std = 0.2393, Count = 954


2

(1732,)
(1022,)
(974,)
(1049,)
PID: ID5E_BDC
  Class 1: Mean Fat Ratio = 0.3285, Std = 0.2520, Count = 1732
  Class 2: Mean Fat Ratio = 0.1168, Std = 0.1695, Count = 1022
  Class 3: Mean Fat Ratio = 0.1040, Std = 0.1692, Count = 974
  Class 4: Mean Fat Ratio = 0.2025, Std = 0.2332, Count = 1049


50: Processing ID21W_R13_L4-L5_sam2_seg_slice17_seg.nii.gz
(1780,)
(967,)
(1969,)
(78,)
PID: ID21W_R13
  Class 1: Mean Fat Ratio = 0.1456, Std = 0.2093, Count = 1780
  Class 2: Mean Fat Ratio = 0.0704, Std = 0.0853, Count = 967
  Class 3: Mean Fat Ratio = 0.0986, Std = 0.1433, Count = 1969
  Class 4: Mean Fat Ratio = 0.0443, Std = 0.0286, Count = 78


51: Processing ID16R_R0_L4-L5_sam2_seg_slice5_seg.nii.gz
(1234,)
(388,)
(655,)
(575,)
PID: ID16R_R0
  Class 1: Mean Fat Ratio = 0.1508, Std = 0.2030, Count = 1234
  Class 2: Mean Fat Ratio = 0.1391, Std = 0.2031, Count = 388
  Class 3: Mean Fat Ratio = 0.1639, Std = 0.2072, Count = 655
  Class 4: Mean Fat Ratio = 0.1946, Std = 0.2479, Count = 575

In [4]:
print(df_results.shape)
df_results.head()



(288, 5)


PID Class  Mean Fat Ratio  Std Fat Ratio Pixel Count
0    ID9J_R0     1        0.193751       0.239271        1349
1    ID9J_R0     2        0.072969       0.116880         983
2    ID9J_R0     3        0.076359       0.123573         958
3    ID9J_R0     4        0.230757       0.253208        1465
4  ID24Z_BDC     1        0.474832       0.262861        1510

In [5]:
import os
import pandas as pd
from scipy.stats import shapiro, ttest_ind, ks_2samp

# -------------- 读取 CSV --------------
data_dir = './videos/bed_rest_MRI'
auto_seg_csv = os.path.join(data_dir,  prompt_dir, prompt_dir+"fat_ratio_results.csv")
label_seg_csv = os.path.join(data_dir, 'label_72_fat_ratio_results.csv')


auto_df = pd.read_csv(auto_seg_csv)
label_df = pd.read_csv(label_seg_csv)

manual_prompt_id = ['ID4D_BDC', 'ID12M_R13', 'ID15Q_R13']
remove_prefixes = manual_prompt_id[:1 + ("1" in prompt_dir) + ("2" in prompt_dir)]  # 根据 prompt_dir 选择要去除的前缀

filter_pid = lambda df: df[~df["PID"].astype(str).str.startswith(tuple(remove_prefixes))]

auto_df, label_df = map(filter_pid, (auto_df, label_df))

print('df,shape', auto_df.shape)


# -------------- 确保 PID 和 Class 匹配 --------------
merged_df = auto_df.merge(label_df, on=["PID", "Class"], suffixes=("_auto", "_label"))

# -------------- 统计学检验 --------------
for class_value in [1, 2, 3, 4]:
    class_data = merged_df[merged_df["Class"] == class_value]

    if len(class_data) < 2:  # 统计学检验至少需要 2 组数据
        print(f"Class {class_value}: Skipping due to insufficient samples.")
        continue

    # 提取脂肪比率
    auto_values = class_data["Mean Fat Ratio_auto"]
    label_values = class_data["Mean Fat Ratio_label"]

    # -------------- 正态性检验 --------------
    shapiro_label = shapiro(label_values)
    shapiro_auto_seg = shapiro(auto_values)

    label_is_normal = shapiro_label.pvalue > 0.05
    auto_seg_is_normal = shapiro_auto_seg.pvalue > 0.05

    print(f"\nClass {class_value}:")
    print(f"  Shapiro-Wilk Normality Test (Label): p-value = {shapiro_label.pvalue:.3f} -> {'Normal' if label_is_normal else 'Non-Normal'}")
    print(f"  Shapiro-Wilk Normality Test (Auto-Seg): p-value = {shapiro_auto_seg.pvalue:.3f} -> {'Normal' if auto_seg_is_normal else 'Non-Normal'}")

    # -------------- 选择统计检验方法 --------------
    if label_is_normal and auto_seg_is_normal:
        t_stat, p_value = ttest_ind(label_values, auto_values, equal_var=False)
        print("  Method: Independent Two-Sample T-Test")
        print(f"  T-statistic: {t_stat:.4f}, P-value: {p_value:.3f}")
    else:
        ks_stat, p_value = ks_2samp(label_values, auto_values)
        print("  Method: Kolmogorov-Smirnov Test")
        print(f"  KS-statistic: {ks_stat:.4f}, P-value: {p_value:.3f}")

    # -------------- 解释检验结果 --------------
    if p_value < 0.05:
        print("  - Significant difference detected (p < 0.05).")
    else:
        print("  - No significant difference detected (p >= 0.05).")


df,shape (284, 5)

Class 1:
  Shapiro-Wilk Normality Test (Label): p-value = 0.000 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.002 -> Non-Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.2817, P-value: 0.007
  - Significant difference detected (p < 0.05).

Class 2:
  Shapiro-Wilk Normality Test (Label): p-value = 0.289 -> Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.000 -> Non-Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.2535, P-value: 0.020
  - Significant difference detected (p < 0.05).

Class 3:
  Shapiro-Wilk Normality Test (Label): p-value = 0.084 -> Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.000 -> Non-Normal
  Method: Kolmogorov-Smirnov Test
  KS-statistic: 0.2113, P-value: 0.084
  - No significant difference detected (p >= 0.05).

Class 4:
  Shapiro-Wilk Normality Test (Label): p-value = 0.000 -> Non-Normal
  Shapiro-Wilk Normality Test (Auto-Seg): p-value = 0.000 -> Non-Normal
  Method: Kolmogoro

In [6]:
for class_value in [2]:
    class_data = merged_df[merged_df["Class"] == class_value]

    if len(class_data) < 2:  # 统计学检验至少需要 2 组数据
        print(f"Class {class_value}: Skipping due to insufficient samples.")
        continue

    # 提取脂肪比率
    auto_values = class_data["Mean Fat Ratio_auto"]
    label_values = class_data["Mean Fat Ratio_label"]

In [7]:
import pandas as pd

# 遍历 Class 2
for class_value in [2]:
    class_data = merged_df[merged_df["Class"] == class_value]

    if len(class_data) < 2:  # 统计学检验至少需要 2 组数据
        print(f"Class {class_value}: Skipping due to insufficient samples.")
        continue

    # 提取脂肪比率
    auto_values = class_data["Mean Fat Ratio_auto"]
    label_values = class_data["Mean Fat Ratio_label"]

    # 计算误差（绝对差值）
    class_data["Error"] = abs(auto_values - label_values)

    # 按误差排序，找到 Top 5
    top_5_diff = class_data.nlargest(10, "Error")

    # 提取对应的 PID 并打印
    top_5_pids = top_5_diff["PID"].tolist()
    print(f"Top 5 PIDs with largest auto-label difference in Class {class_value}: {top_5_pids}")


Top 5 PIDs with largest auto-label difference in Class 2: ['ID24Z_R0', 'ID22X_R0', 'ID5E_R0', 'ID12M_R0', 'ID14P_BDC', 'ID18T_R13', 'ID13N_R0', 'ID1A_R13', 'ID8H_R0', 'ID12M_BDC']


/tmp/ipykernel_26939/2447707410.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_data["Error"] = abs(auto_values - label_values)


In [8]:
top_5_diff

PID  Class  Mean Fat Ratio_auto  Std Fat Ratio_auto  \
169   ID24Z_R0      2             0.132876            0.131042   
257   ID22X_R0      2             0.096748            0.112901   
65     ID5E_R0      2             0.187836            0.231431   
109   ID12M_R0      2             0.089659            0.107151   
145  ID14P_BDC      2             0.130681            0.179652   
233  ID18T_R13      2             0.048688            0.045871   
53    ID13N_R0      2             0.113776            0.156685   
241   ID1A_R13      2             0.137505            0.189048   
33     ID8H_R0      2             0.138522            0.157455   
13   ID12M_BDC      2             0.077999            0.103663   

     Pixel Count_auto  Mean Fat Ratio_label  Std Fat Ratio_label  \
169               583              0.269814             0.283529   
257               568              0.171117             0.205138   
65               1235              0.114285             0.146913   
109               542              0.154643             0.199711   
145               575              0.193881             0.238419   
233                90              0.111322             0.146223   
53                729              0.174124             0.223099   
241              1184              0.077465             0.106815   
33                431              0.194802             0.236892   
13                639              0.131159             0.176730   

     Pixel Count_label     Error  
169                835  0.136938  
257                623  0.074368  
65                 906  0.073551  
109                666  0.064983  
145                707  0.063199  
233                622  0.062633  
53                 612  0.060348  
241                926  0.060040  
33                 500  0.056280  
13                 795  0.053160

In [9]:
auto_values = list(auto_values)
label_values  =  list(label_values )